<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Library-Imports" data-toc-modified-id="Library-Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Library Imports</a></span></li><li><span><a href="#Cuda-Support" data-toc-modified-id="Cuda-Support-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cuda Support</a></span></li></ul></li><li><span><a href="#CartPole-v1-Environment" data-toc-modified-id="CartPole-v1-Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>CartPole v1 Environment</a></span><ul class="toc-item"><li><span><a href="#Action/Input-Space" data-toc-modified-id="Action/Input-Space-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Action/Input Space</a></span></li><li><span><a href="#Observation/Output-Space" data-toc-modified-id="Observation/Output-Space-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Observation/Output Space</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#Informational-Functions" data-toc-modified-id="Informational-Functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Informational Functions</a></span></li><li><span><a href="#Agent-Factory" data-toc-modified-id="Agent-Factory-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Agent Factory</a></span></li><li><span><a href="#Simulation-Function" data-toc-modified-id="Simulation-Function-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Simulation Function</a></span></li></ul></li><li><span><a href="#Random-Agent" data-toc-modified-id="Random-Agent-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Random Agent</a></span></li><li><span><a href="#One-Layer-Neural-Nets" data-toc-modified-id="One-Layer-Neural-Nets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>One Layer Neural Nets</a></span><ul class="toc-item"><li><span><a href="#BNN-Neural-Net-Definition" data-toc-modified-id="BNN-Neural-Net-Definition-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>BNN Neural Net Definition</a></span></li><li><span><a href="#BNN_1-Parameters" data-toc-modified-id="BNN_1-Parameters-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>BNN_1 Parameters</a></span></li><li><span><a href="#BNN_1-Class" data-toc-modified-id="BNN_1-Class-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>BNN_1 Class</a></span></li><li><span><a href="#BNN_1-simulation" data-toc-modified-id="BNN_1-simulation-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>BNN_1 simulation</a></span></li><li><span><a href="#BNN_2-Parameters" data-toc-modified-id="BNN_2-Parameters-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>BNN_2 Parameters</a></span></li><li><span><a href="#BNN_2-Definition" data-toc-modified-id="BNN_2-Definition-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>BNN_2 Definition</a></span></li><li><span><a href="#BNN_2-simulation" data-toc-modified-id="BNN_2-simulation-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>BNN_2 simulation</a></span></li></ul></li><li><span><a href="#Two-Layer-Neural-Nets" data-toc-modified-id="Two-Layer-Neural-Nets-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Two Layer Neural Nets</a></span><ul class="toc-item"><li><span><a href="#TLN-With-Tanh,-Neural-Net-Definition" data-toc-modified-id="TLN-With-Tanh,-Neural-Net-Definition-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>TLN With Tanh, Neural Net Definition</a></span></li><li><span><a href="#TLN_1-Parameters" data-toc-modified-id="TLN_1-Parameters-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>TLN_1 Parameters</a></span></li><li><span><a href="#TLN_1-Simulation" data-toc-modified-id="TLN_1-Simulation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>TLN_1 Simulation</a></span></li></ul></li><li><span><a href="#Random-Cartpole" data-toc-modified-id="Random-Cartpole-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Random Cartpole</a></span></li></ul></div>

## Setup

### Library Imports

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from time import time
import gym
import math

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

### Cuda Support

In [2]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Tensor = torch.Tensor

## CartPole v1 Environment 

In [3]:
env = gym.make('CartPole-v1')
inputs = env.observation_space.shape[0]
outputs = env.action_space.n

print("Input Space:\t{}\nOutput Space:\t{}".format(inputs, outputs))
score_to_solve = 500

Input Space:	4
Output Space:	2


### Action/Input Space

|Num|Action|
|-|-|
|0|Push cart to the left|
|1|Push cart to the right|

### Observation/Output Space     


|Num|Observation|Min|Max|
|-|---------------------|-|-|
|0|Cart Position|-4.8|4.8|
|1|Cart Velocity|-Inf|Inf|
|2|Pole Angle|-24 deg|24 deg|
|3|Pole Velocity At Tip|-Inf|Inf|


## Functions

### Informational Functions

In [4]:
def plot_information(steps_total, shadow_run):
    e_list = []
    for i in shadow_run:
        for j in shadow_run[i]["epsilon"]:
            e_list.append(j)
    
    #s = list(range(sum(steps_total)))
    
    fig, ax = plt.subplots(figsize=(12,10), nrows=2, ncols=1)
    fig.subplots_adjust(hspace=.25)

    ax[0].set(title="Rewards",
              xlabel='Session', 
              ylabel='Steps')
    ax[0].bar(torch.arange(len(steps_total)), 
            steps_total, 
            alpha=0.6, 
            color='green',
            width= 1)
    
    ax[1].set(title="Epsilon",
          xlabel='Steps', 
          ylabel='Session')
    ax[1].plot(e_list)
    plt.show()
    
def report(report_interval,
           episode,
           start_time,
           steps_total,
           epsilon,
           frames_total,
           solved):
    
    if episode % report_interval == 0 and episode != 0:
        print("""\t\t\t *** Episode {} at Total Seconds {:2f}***
                 Average Reward for last {} steps: {:2f}
                 Average Reward for all steps: {:2f}
                 Epsilon: {}, Frames Total :{}
                 Solved {} Times
              """.format(episode,
                   (time() - start_time),
                   str(report_interval),
                   sum(steps_total[-report_interval:])/report_interval,
                   sum(steps_total)/len(steps_total),
                   epsilon,
                   frames_total,
                   solved))

### Agent Factory

In [5]:
class Agent(object):
  
    def factory(type):
        if type == "Random":
            return Random_Agent(type)
        elif type == "BNN_1":
            return BNN_1(type)
        elif type == "BNN_2":
            return BNN_2(type)
        elif type == "TLN_1":
            return TLN_1(type)
        
    factory = staticmethod(factory)

### Simulation Function

In [6]:
def simulation(num_episodes, agent, record, report_interval, extra_name  = ""):
    #Simulation Initialization
    actor = Agent.factory(agent)
    start_time = time()
    solved, frames_total = 0, 0
    steps_total = []
    
    if extra_name != "": extra_name = "_" + extra_name
    
    #The recorded dictionary for the simulation
    shadow_run = {}
    
    #Loop through different episodes
    for episode in range(num_episodes):

        # Episode Initializations
        state = env.reset()
        epi_name = actor.name + extra_name + "_"+ str(episode).zfill(4)
        shadow_run[epi_name] = {} 
        
        #A facny way to make lists
        blanks = [-1 for i in range(500)]
        
        #Input for episode
        actions, steps = [blanks for i in range(2)] 
        
        #Outputs for episodes
        cart_position, cart_velocity = [blanks for i in range(2)]
        pole_angle, pole_velocity  = [blanks for i in range(2)]
        
        step = 0
                
        while True:
            
            #Calculate Learning Rate
            epsilon = actor.calculate_epsilon(frames_total)

            #Actor acts
            action = actor.select_action(state, epsilon)

            #Record States
            new_state, reward, done, info = env.step(action)

            #Optimize
            actor.optimize(state, action, new_state, reward, done)

            #Store Inputs for step
            actions[step] = action
            
            ###
            #**DO THIS**
            #On update, make a class called Information to clean syntax
            ###
            
            #Unpack Output for step
            cart_position[step] = new_state[0]
            cart_velocity[step] = new_state[1]
            pole_angle[step] = new_state[2]
            pole_velocity[step] = new_state[3]         

            if done:
                steps_total.append(step)
                
                ###
                #**DO THIS**
                #On update, clean up the following syntax
                ###
                
                #Below is an overpacked syntax to add a column for each part
                #.zfill pads zeros in the front of the string
                
                for i in range(500):
                    D = shadow_run[epi_name]
                    D["action_{}".format(str(i).zfill(3))]        = actions[i]
                    D["cart_position_{}".format(str(i).zfill(3))] = cart_position[i]
                    D["cart_velocity_{}".format(str(i).zfill(3))] = cart_velocity[i]
                    D["pole_angle_{}".format(str(i).zfill(3))]    = pole_angle[i]
                    D["pole_velocity_{}".format(str(i).zfill(3))] = pole_velocity[i]                                     
                
                shadow_run[epi_name] = D
                #Not needed for this environment
                #shadow_run[episode]["information"] = information

                if step >= score_to_solve: solved += 1

                report(report_interval, episode, start_time, steps_total, epsilon, frames_total, solved)

                break

            step += 1
            frames_total += 1
            state = new_state
        
    #Clean up after Episodes
    record[actor.name + extra_name] = shadow_run    
    env.close()
    env.env.close()

    return record

## Random Agent

In [7]:
class Random_Agent(Agent):
    def __init__(self, Agent):
        self.name = Agent
        
    def select_action(self,state,epsilon):
        action = env.action_space.sample()
        return action
    
    def calculate_epsilon(self,steps_done):
        return 0    
    
    def optimize(self, state, action, new_state, reward, done):
        return 0

In [8]:
record = simulation(1000, "Random", {}, 100)

			 *** Episode 100 at Total Seconds 0.142594***
                 Average Reward for last 100 steps: 22.740000
                 Average Reward for all steps: 22.811881
                 Epsilon: 0, Frames Total :2304
                 Solved 0 Times
              
			 *** Episode 200 at Total Seconds 0.260123***
                 Average Reward for last 100 steps: 19.680000
                 Average Reward for all steps: 21.253731
                 Epsilon: 0, Frames Total :4272
                 Solved 0 Times
              
			 *** Episode 300 at Total Seconds 0.381009***
                 Average Reward for last 100 steps: 23.630000
                 Average Reward for all steps: 22.043189
                 Epsilon: 0, Frames Total :6635
                 Solved 0 Times
              
			 *** Episode 400 at Total Seconds 0.501348***
                 Average Reward for last 100 steps: 20.490000
                 Average Reward for all steps: 21.655860
                 Epsilon: 0, Frames Total :

In [9]:
len(record["Random"].keys())

1000

In [10]:
pd.DataFrame.from_dict(record["Random"]).T.columns

Index(['action_000', 'action_001', 'action_002', 'action_003', 'action_004',
       'action_005', 'action_006', 'action_007', 'action_008', 'action_009',
       ...
       'pole_velocity_490', 'pole_velocity_491', 'pole_velocity_492',
       'pole_velocity_493', 'pole_velocity_494', 'pole_velocity_495',
       'pole_velocity_496', 'pole_velocity_497', 'pole_velocity_498',
       'pole_velocity_499'],
      dtype='object', length=2500)

In [11]:
pd.DataFrame.from_dict(record["Random"]).T.head()

,action_000,action_001,action_002,action_003,action_004,action_005,action_006,action_007,action_008,action_009,...,pole_velocity_490,pole_velocity_491,pole_velocity_492,pole_velocity_493,pole_velocity_494,pole_velocity_495,pole_velocity_496,pole_velocity_497,pole_velocity_498,pole_velocity_499
Random_0000,-0.310608,-0.023597,0.261409,-0.038900,-0.337600,-0.051307,0.232801,-0.068528,0.216755,-0.083568,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Random_0001,0.302538,0.004005,-0.292654,-0.589287,-0.302476,-0.604543,-0.323510,-0.046471,-0.355945,-0.081362,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Random_0002,-0.274389,0.021212,-0.270266,0.023749,0.316062,0.608525,0.317619,0.615861,0.330880,0.049916,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Random_0003,-0.325925,-0.043888,-0.348827,-0.069210,-0.376509,-0.099649,0.174698,0.448381,0.723243,1.001101,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Random_0004,0.314862,0.019873,-0.273140,0.019338,0.310087,0.600959,0.893792,0.605063,0.907156,1.212880,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## One Layer Neural Nets

### BNN Neural Net Definition

In [12]:
class NeuralNetwork_1(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(number_of_inputs,number_of_outputs)
        
    def forward(self, x):
        output = self.linear1(x)
        return output

### BNN_1 Parameters

In [13]:
###### PARAMS ######
learning_rate = 0.025
num_episodes = 1000
gamma = 0.85

egreedy = 0.9
egreedy_final = 0.025
egreedy_decay = 500

####################

#These lines are useful if I decide to change environments
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n

### BNN_1 Class

In [14]:
class BNN_1(object):
    
    def __init__(self,Agent):
        self.nn = NeuralNetwork_1().to(device)
        self.name = Agent

        self.loss_func = nn.MSELoss()
        #self.loss_func = nn.SmoothL1Loss()
        
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        #self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)
    
    def calculate_epsilon(self, steps_done):
        epsilon = egreedy_final + (egreedy - egreedy_final) * \
                  math.exp(-1. * steps_done / egreedy_decay )
        return epsilon
    
    def select_action(self,state,epsilon):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > epsilon:      
            
            with torch.no_grad():
                
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()        
        else:
            action = env.action_space.sample()
        
        return action
    
    def optimize(self, state, action, new_state, reward, done):
        
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        
        reward = Tensor([reward]).to(device)
        
        if done:
            target_value = reward
        else:
            new_state_values = self.nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values)
            target_value = reward + gamma * max_new_state_values
        
        predicted_value = self.nn(state)[action]
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

### BNN_1 simulation

In [15]:
record = simulation(1000, "BNN_1", record, 100)

			 *** Episode 100 at Total Seconds 0.596590***
                 Average Reward for last 100 steps: 12.010000
                 Average Reward for all steps: 11.990099
                 Epsilon: 0.10265095795554963, Frames Total :1211
                 Solved 0 Times
              
			 *** Episode 200 at Total Seconds 1.175857***
                 Average Reward for last 100 steps: 11.830000
                 Average Reward for all steps: 11.910448
                 Epsilon: 0.03228796156610423, Frames Total :2394
                 Solved 0 Times
              
			 *** Episode 300 at Total Seconds 1.627053***
                 Average Reward for last 100 steps: 8.790000
                 Average Reward for all steps: 10.873754
                 Epsilon: 0.026256366577283762, Frames Total :3273
                 Solved 0 Times
              
			 *** Episode 400 at Total Seconds 2.276345***
                 Average Reward for last 100 steps: 15.030000
                 Average Reward for all steps:

In [16]:
record.keys()

dict_keys(['Random', 'BNN_1'])

In [17]:
len(record["BNN_1"].keys())

1000

### BNN_2 Parameters

In [18]:
learning_rate = 0.001
num_episodes = 1000
gamma = 0.85

egreedy = 0.9
egreedy_final = 0.01
egreedy_decay = 750

#These lines are useful if I decide to change environments
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n

### BNN_2 Definition

In [19]:
#I might want to tinker more with these settings later, so I am copying it 
#From BNN_1 for now.

###
#**DO THIS**
#Next time consider either significantly changing code
#Or, simply duplicate the BNN_1 class
###

class BNN_2(object):
    
    def __init__(self,Agent):
        self.nn = NeuralNetwork_1().to(device)
        self.name = Agent

        self.loss_func = nn.MSELoss()
        #self.loss_func = nn.SmoothL1Loss()
        
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        #self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)
    
    def calculate_epsilon(self, steps_done):
        epsilon = egreedy_final + (egreedy - egreedy_final) * \
                  math.exp(-1. * steps_done / egreedy_decay )
        return epsilon
    
    def select_action(self,state,epsilon):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > epsilon:      
            
            with torch.no_grad():
                
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()        
        else:
            action = env.action_space.sample()
        
        return action
    
    def optimize(self, state, action, new_state, reward, done):
        
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        
        reward = Tensor([reward]).to(device)
        
        if done:
            target_value = reward
        else:
            new_state_values = self.nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values)
            target_value = reward + gamma * max_new_state_values
        
        predicted_value = self.nn(state)[action]
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

### BNN_2 simulation

In [20]:
record = simulation(1000, "BNN_2", record, 100)

			 *** Episode 100 at Total Seconds 0.542281***
                 Average Reward for last 100 steps: 11.860000
                 Average Reward for all steps: 11.861386
                 Epsilon: 0.19016770820988002, Frames Total :1198
                 Solved 0 Times
              
			 *** Episode 200 at Total Seconds 0.965622***
                 Average Reward for last 100 steps: 8.610000
                 Average Reward for all steps: 10.243781
                 Epsilon: 0.06716193068769614, Frames Total :2059
                 Solved 0 Times
              
			 *** Episode 300 at Total Seconds 1.394936***
                 Average Reward for last 100 steps: 8.650000
                 Average Reward for all steps: 9.714286
                 Epsilon: 0.02803933760906884, Frames Total :2924
                 Solved 0 Times
              
			 *** Episode 400 at Total Seconds 1.802954***
                 Average Reward for last 100 steps: 8.450000
                 Average Reward for all steps: 9.3

In [21]:
record.keys()

dict_keys(['Random', 'BNN_1', 'BNN_2'])

## Two Layer Neural Nets

### TLN With Tanh, Neural Net Definition

In [39]:
class NeuralNetwork_2(nn.Module):
    def __init__(self):
        super(NeuralNetwork_2, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer)
        self.linear2 = nn.Linear(hidden_layer,number_of_outputs)

        self.activation = nn.Tanh()
        #self.activation = nn.ReLU()
        
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)
        
        return output2

### TLN_1 Parameters

In [40]:
learning_rate = 0.01
gamma = 0.99

hidden_layer = 64

egreedy = 0.9
egreedy_final = 0.02
egreedy_decay = 500

number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n

In [41]:
class TLN_1(object):
    
    def __init__(self,Agent):
        self.nn = NeuralNetwork_2().to(device)
        self.name = Agent

        self.loss_func = nn.MSELoss()
        #self.loss_func = nn.SmoothL1Loss()
        
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        #self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)
    
    def calculate_epsilon(self, steps_done):
        epsilon = egreedy_final + (egreedy - egreedy_final) * \
                  math.exp(-1. * steps_done / egreedy_decay )
        return epsilon
    
    def select_action(self,state,epsilon):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > epsilon:      
            
            with torch.no_grad():
                
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()        
        else:
            action = env.action_space.sample()
        
        return action
    
    def optimize(self, state, action, new_state, reward, done):
        
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        
        reward = Tensor([reward]).to(device)
        
        if done:
            target_value = reward
        else:
            new_state_values = self.nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values)
            target_value = reward + gamma * max_new_state_values
        
        predicted_value = self.nn(state)[action]
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

### TLN_1 Simulation

In [42]:
record = simulation(1000, "TLN_1", record, 100)

			 *** Episode 100 at Total Seconds 5.835294***
                 Average Reward for last 100 steps: 129.040000
                 Average Reward for all steps: 127.920792
                 Epsilon: 0.020000000005276103, Frames Total :12920
                 Solved 0 Times
              
			 *** Episode 200 at Total Seconds 17.452231***
                 Average Reward for last 100 steps: 259.400000
                 Average Reward for all steps: 193.333333
                 Epsilon: 0.02, Frames Total :38860
                 Solved 0 Times
              
			 *** Episode 300 at Total Seconds 31.296522***
                 Average Reward for last 100 steps: 281.410000
                 Average Reward for all steps: 222.594684
                 Epsilon: 0.02, Frames Total :67001
                 Solved 0 Times
              
			 *** Episode 400 at Total Seconds 45.813516***
                 Average Reward for last 100 steps: 270.710000
                 Average Reward for all steps: 234.593516
    

In [ ]:
learning_rate = 0.02
gamma = 0.99

egreedy = 0.9
egreedy_final = 0.001
egreedy_decay = num_episodes * 5
report_interval = 10

hidden_layer = 64

In [ ]:
qnet_agent = QNet_Agent()
start_time = time()
solved, frames_total = 0, 0
steps_total = []
shadow_run = {}

for episode in range(num_episodes):
    # Episode Initializations
    state = env.reset()
    shadow_run[episode] = {}
    actions, states, shots = [], [], []
    step = 0

    while True:
        step += 1
        frames_total += 1

        epsilon = calculate_epsilon(frames_total)

        #action = env.action_space.sample()
        action = qnet_agent.select_action(state, epsilon)

        new_state, reward, done, info = env.step(action)
        rendered = env.render(mode = 'rgb_array')

        # Agent Optimize
        qnet_agent.optimize(state, action, new_state, reward, done)

        # Store states for step
        actions.append(action)
        states.append(new_state)
        shots.append(rendered)

        if done:
            steps_total.append(step)
            shadow_run[episode]["actions"] = actions
            shadow_run[episode]["states"] = states
            shadow_run[episode]["shots"] = shots
            shadow_run[episode]["steps"] = step

            if step >= score_to_solve: solved += 1

            report(report_interval, episode, start_time, steps_total, epsilon, frames_total, solved)

            break

        state = new_state

env.close()
env.env.close()

plot_information(steps_total,shadow_run)
display_frames(num_episodes-1)

In [ ]:
class Car(object):
  
    def factory(type):
        if type == "Racecar":
            return Racecar(type)
        if type == "Van":
            return Van(type)

    factory = staticmethod(factory)

class Racecar(Car):
    
    def __init__(self,Car):
        self.name = Car
        
    def drive(self):
        print("Racecar driving.")

class Van(Car):
    def drive(self):
        print("Van driving.")

# Create object using factory.
obj = Car.factory("Racecar")
obj.name

## Random Cartpole

In [ ]:
class Agent(object,name):
    def __init__(self,name):
        
        self.name = name
        
    
    
    
    def __init__(self):
        
        self.name = name
        
        if self.name == "random":
        else:
            self.nn = NeuralNetwork().to(device)
            self.loss_func = nn.MSELoss()
            #self.loss_func = nn.SmoothL1Loss()
            self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
            #self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)
        
    def select_action(self,state,epsilon):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > epsilon:      
            
            with torch.no_grad():
                
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()        
        else:
            action = env.action_space.sample()
        
        return action
    
    def optimize(self, state, action, new_state, reward, done):
        
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        
        reward = Tensor([reward]).to(device)
        
        if done:
            target_value = reward
        else:
            new_state_values = self.nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values)
            target_value = reward + gamma * max_new_state_values
        
        predicted_value = self.nn(state)[action]
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
num_episodes = 1000

steps_total = []

for i_episode in range(num_episodes):
    
    state = env.reset()    
    step = 0
    #for step in range(100):
    while True:
        
        step += 1
        
        #Random Step
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        
        if done:
            
            steps_total.append(step)
            print("Episode finished after %i steps" % step )
            break
        

print("Average reward: %.2f" % (sum(steps_total)/num_episodes))
print("Average reward (last 100 episodes): %.2f" % (sum(steps_total[-100:])/100))

plt.figure(figsize=(12,5))
plt.title("Rewards")
plt.bar(torch.arange(len(steps_total)), steps_total, alpha=0.6, color='green')
plt.show()

env.close()
env.env.close()

In [ ]:
test["Random"]

In [ ]:
blanks = [-200 for i in range(500)]
actions, states, information = [blanks for i in range(3)]

In [ ]:
actions

In [ ]:
bob = [0,0,0]

In [ ]:
bob[2] = 34

In [ ]:
bob

In [ ]:
str(54).zfill(3)